# ECC Assignement 2 - Spark

In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.context import SparkContext, SparkConf
from pyspark.sql.context import SQLContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType

import pandas as pd
import requests
import json

In [2]:
spark = SparkSession.builder \
      .master("local[2]") \
      .appName("SparkByExamples.com") \
      .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/12 09:08:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/12 09:08:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## NYC Parking Violation Dataset

In [3]:
df_nyc = spark.read.format("csv").load("Parking_Violations_Issued_-_Fiscal_Year_2023.csv")

In [4]:
df_nyc.take(1)

22/12/12 09:08:43 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(_c0='Summons Number', _c1='Plate ID', _c2='Registration State', _c3='Plate Type', _c4='Issue Date', _c5='Violation Code', _c6='Vehicle Body Type', _c7='Vehicle Make', _c8='Issuing Agency', _c9='Street Code1', _c10='Street Code2', _c11='Street Code3', _c12='Vehicle Expiration Date', _c13='Violation Location', _c14='Violation Precinct', _c15='Issuer Precinct', _c16='Issuer Code', _c17='Issuer Command', _c18='Issuer Squad', _c19='Violation Time', _c20='Time First Observed', _c21='Violation County', _c22='Violation In Front Of Or Opposite', _c23='House Number', _c24='Street Name', _c25='Intersecting Street', _c26='Date First Observed', _c27='Law Section', _c28='Sub Division', _c29='Violation Legal Code', _c30='Days Parking In Effect    ', _c31='From Hours In Effect', _c32='To Hours In Effect', _c33='Vehicle Color', _c34='Unregistered Vehicle?', _c35='Vehicle Year', _c36='Meter Number', _c37='Feet From Curb', _c38='Violation Post Code', _c39='Violation Description', _c40='No Standing o

## Question 1

### When are tickets most likely to be issued?



The question states at what time of the day the tickets are issued and total number of tickets issued at that particular time 

To solve the question we just use the groupby function in pyspark on the "Violation time" and count the number of ticket issued at that particular time.

In [5]:
df_nyc_q1 = df_nyc.groupBy("_c19").count()

Once the new dataframe is created of violation time and the count of ticket issued at that paritcular time I arranged it in descending order and display the top 5 time when ticket was issued.

In [6]:
df_nyc_q1 = df_nyc_q1.sort(desc("count"))
print("Top 5 times when ticket are most likely issued")
df_nyc_q1 = df_nyc_q1.select(col("_c19").alias("Time when ticket was issued"), col("count").alias("Total number of tickets")).show(5)

Top 5 times when ticket are most likely issued


+---------------------------+-----------------------+
|Time when ticket was issued|Total number of tickets|
+---------------------------+-----------------------+
|                      0836A|                  12003|
|                      0906A|                  11394|
|                      0838A|                  11281|
|                      0839A|                  11270|
|                      0840A|                  11202|
+---------------------------+-----------------------+
only showing top 5 rows



The output shown above lists the top 5 situations in NYC when(time) a ticket is be issued. It displays two columns: one for the violation time, or "when," the ticket is issued, and the other for the violation count, or the total number of tickets issued at that time. According to this table, we may deduce that the most tickets were probably issued at 0836A with a count of 12003.

## Question 2
### What are the most common years and types of cars to be ticketed?

The question states that, the year or "Vehicle Year" and the type or "Vehicle Type" acting like a single parameter are the most ticketed 

To solve the question I calculated the count of Vehicle Type and Vehicle Year as a single parameter

In [14]:
df_nyc_q2 = df_nyc.na.drop(subset=["_c6"]).withColumn("_c35", df_nyc["_c35"].cast(IntegerType())).filter("_c35 > 0")
df_nyc_q2 = df_nyc_q2.groupBy("_c6", "_c35").count()

Once the count is done I sorted the new table in descending order and displayed the top 20 Vehicle type and Year

In [15]:
df_nyc_q2 = df_nyc_q2.sort(desc("count"))
print("Top 20 most common Vehicle Type and Years: ")
df_nyc_q2 = df_nyc_q2.select(col("_c6").alias("Vehicle Type"), col("_c35").alias("Vehicle Year"), col("count").alias("Violation Count")).show(20)

Top 20 most common Vehicle Type and Years: 


+------------+------------+---------------+
|Vehicle Type|Vehicle Year|Violation Count|
+------------+------------+---------------+
|        SUBN|        2021|         302579|
|        SUBN|        2020|         254398|
|        SUBN|        2019|         229424|
|        SUBN|        2022|         213913|
|        SUBN|        2018|         177057|
|        SUBN|        2017|         144410|
|        SUBN|        2016|         116255|
|        4DSD|        2017|         111780|
|        SUBN|        2015|         110579|
|        4DSD|        2018|         109220|
|        4DSD|        2019|         106716|
|        4DSD|        2020|          98466|
|        4DSD|        2021|          88997|
|        4DSD|        2015|          88408|
|        4DSD|        2016|          88097|
|        SUBN|        2014|          84805|
|        SUBN|        2013|          78531|
|        4DSD|        2022|          71645|
|        SUBN|        2011|          71071|
|        4DSD|        2013|     

The output shown above lists the top 20 years and car types for which tickets were issued. The first column in this table lists the kind of the vehicle that received a ticket, the second lists the typical years when tickets were issued, and the third gives the total number of tickets issued in relation to the first two columns. We deduce from the data that the SUBN vehicle type received tickets 302579 times  in 2021.

## Question 3 
### Where are tickets most commonly issued? 


The question states at what location of the day the tickets are issued and total number of tickets issued at that particular location


To solve the question we just use the groupby function in pyspark on the "Violation location" and count the number of ticket issued at that particular location

In [9]:
df_nyc_q3 = df_nyc.na.drop(subset=["_c13"])
df_nyc_q3 = df_nyc_q3.groupBy("_c13").count()

Once the new dataframe is created of violation lcoation and the count of ticket issued at that paritcular location I arranged it in descending order and display the top 10 location when ticket was issued.

In [10]:
df_nyc_q3 = df_nyc_q3.sort(desc("count"))
print("Top 10 most common Violation Location: ")
df_nyc_q3 = df_nyc_q3.select(col("_c13").alias("Violation Location"), col("count").alias("Violation Count")).show(10)

Top 10 most common Violation Location: 


+------------------+---------------+
|Violation Location|Violation Count|
+------------------+---------------+
|              0013|         146695|
|              0019|         142982|
|              0006|         119458|
|               114|         111627|
|              0014|         102973|
|              0018|          99891|
|              0009|          86819|
|              0001|          86580|
|               108|          64908|
|              0020|          63996|
+------------------+---------------+
only showing top 10 rows



The output shown above lists the top 10 situations in NYC where(location) a ticket is be issued. It displays two columns: one for the violation location, or "where," the ticket is issued, and the other for the violation count, or the total number of tickets issued at that location. According to this table, we may deduce that the most tickets were probably issued at 0013 with a count of 146695.

## Question 4
### Which color of the vehicle is most likely to get a ticket? 

The question states which color of the vehicle is most likely to get a ticket

To solve the question we just use the groupby function in pyspark on the "Vehicle Color" and count the number of ticket issued for that particular color

In [16]:
df_nyc_q4 = df_nyc.na.drop(subset=["_c33"])
df_nyc_q4 = df_nyc_q4.groupBy("_c33").count()

Once the new dataframe is created of Vehicle Color and the count of ticket issued for that paritcular color, I arranged it in descending order and display the top 10 vehicle color for which the ticket was issued.

In [12]:
df_nyc_q4 = df_nyc_q4.sort(desc("count"))
print("Top 10 most common Vehicle Color to get a ticket: ")
df_nyc_q4 = df_nyc_q4.select(col("_c33").alias("Vehicle Color"), col("count").alias("Violation Count")).show(10)

Top 10 most common Vehicle Color to get a ticket: 


+-------------+---------------+
|Vehicle Color|Violation Count|
+-------------+---------------+
|           GY|        1204214|
|           WH|        1100147|
|           BK|        1056412|
|           BL|         404706|
|        WHITE|         363462|
|           RD|         233608|
|        BLACK|         221080|
|         GREY|         157773|
|         BLUE|          81232|
|        BROWN|          79706|
+-------------+---------------+
only showing top 10 rows



The output shown above lists the top 10 vehicle color in NYC for which a ticket is be issued. It displays two columns: one for the vehicle color, or "which color," the ticket is issued, and the other for the violation count, or the total number of tickets issued for that particular color. According to this table, we may deduce that the most tickets were issued for Grey(GY) with a count of 1204214.

## NBA Shots Log File

In [17]:
df_nba = spark.read.format("csv").load("shot_logs.csv")

In [18]:
df_nba.take(1)

[Row(_c0='GAME_ID', _c1='MATCHUP', _c2='LOCATION', _c3='W', _c4='FINAL_MARGIN', _c5='SHOT_NUMBER', _c6='PERIOD', _c7='GAME_CLOCK', _c8='SHOT_CLOCK', _c9='DRIBBLES', _c10='TOUCH_TIME', _c11='SHOT_DIST', _c12='PTS_TYPE', _c13='SHOT_RESULT', _c14='CLOSEST_DEFENDER', _c15='CLOSEST_DEFENDER_PLAYER_ID', _c16='CLOSE_DEF_DIST', _c17='FGM', _c18='PTS', _c19='player_name', _c20='player_id')]

## Question 1 (NBA)

### For each pair of the players (A, B), we define the fear sore of A when facing B is the hit rate, such that B is closet defender when A is shooting. Based on the fear sore, for each player, please find out who is his ”most unwanted defender”

The below step involves the grouping of columns Player ID and Defender ID and counts the total number of hits - 1's and no hits - 0's in these columns.

In [29]:
cnt_cond = lambda cond: sum(when(cond, 1).otherwise(0))
df_nba_q1 = df_nba.groupBy(col("_c20").alias("PlayerID"), col("_c15").alias("DefenderID")) .agg(cnt_cond(col("_c13") == "made").alias("count1s"), cnt_cond(col("_c13") == "missed").alias("count0s"))

The block below involves the calculation of the hit rate. 1's denotes if goes inside the basket hits and 0's denotes no basket. Thus, it follows the formula of hits/total number of hits [hits + no hits]

In [30]:
df_nba_q1 = df_nba_q1.withColumn("HitRate", col=col("count1s")/(col("count1s") + col("count0s")))
df_nba_q1 = df_nba_q1.na.drop(subset=["HitRate"])

The block below drops the duplicates of PlayerID and HitRate

In [31]:
df_nba_q1 = df_nba_q1.dropDuplicates(["PlayerID", "HitRate"])

The block below involves finding the minimum value of hit rate for each Player ID

In [32]:
df_nba_q11 = df_nba_q1.groupBy("PlayerID").agg({"HitRate": "min"}).withColumn("HitRate", col("min(HitRate)"))

Finally, I joined the two columns i.e., Player ID and Defender ID, drop duplicates in them and display the output of Players along with their most unwanted defenders based on the value generated from the Hit Rate.

In [33]:
df_nba_q1 = df_nba_q1.join(df_nba_q11, ["PlayerID", "HitRate"]).select("PlayerID", "DefenderID")
df_nba_q1 = df_nba_q1.join(df_nba, [df_nba["_c20"] == df_nba_q1["PlayerID"], df_nba["_c15"] == df_nba_q1["DefenderID"]]).withColumn("Player Name", col("_c19")).withColumn("Most Unwanted Defender", col("_c14")).dropDuplicates(["PlayerID", "DefenderID"]).select("Player Name", "Most Unwanted Defender").show(10)

+------------------+----------------------+
|       Player Name|Most Unwanted Defender|
+------------------+----------------------+
|      andrew bogut|         Hinrich, Kirk|
|   marvin williams|      Fredette, Jimmer|
|        chris paul|      Carroll, DeMarre|
|charlie villanueva|     Cousins, DeMarcus|
|     channing frye|       Roberson, Andre|
|    deron williams|                  Nene|
|      gerald green|        Leonard, Kawhi|
|     nate robinson|          Williams, Mo|
|      jarrett jack|     Parsons, Chandler|
|     jason maxiell|  Caldwell-Pope, Ke...|
+------------------+----------------------+
only showing top 10 rows



In this problem, the hit rate is used to define each player's fear score while they are facing their opponent, so that the opponent is the player's closest defence when the player is firing. The top 10 most despised defenders are shown in the above table for each player based on their fear score. We can conclude that Kirk Hinrich is the least desired defender for the player Andrew Bogut, and so on.

## Question 2 (NBA)

### Please develop a Spark-based algorithm to classify each player’s records into 4 comfortable zones. Considering the hit rate, which zone is the best for James Harden, Chris Paul,Stephen Curry, and Lebron James

The block below involves the grouping of columns Player ID and Shot Distance, ColestDefDistance and ShotClock and counts the total number of hits - 1's and no hits - 0's in these columns.

In [34]:
cnt_cond = lambda cond: sum(when(cond, 1).otherwise(0))
df_nba_q2 = df_nba.groupBy(col("_c20").alias("PlayerID"), col("_c11").alias("ShotDistance"), col("_c16").alias("ClosestDefDistance"), col("_c8").alias("ShotClock")).agg(cnt_cond(col("_c13") == "made").alias("count1s"), cnt_cond(col("_c13") == "missed").alias("count0s"))

The block below involves the calculation of the hit rate. 1's denotes if the hits and 0's denotes no hits. Thus, it follows the formula of hits/total number of hits [hits + no hits]

In [35]:
df_nba_q2 = df_nba_q2.withColumn("HitRate", col=col("count1s")/(col("count1s") + col("count0s")))
df_nba_q2 = df_nba_q2.na.drop(subset=["HitRate"])

The block below drops the duplicates of PlayerID and HitRate

In [36]:
df_nba_q2 = df_nba_q2.dropDuplicates(["PlayerID", "HitRate"])

The block below involves finding the minimum value of hit rate for each Player ID

In [37]:
df_nba_q22 = df_nba_q2.groupBy("PlayerID").agg({"HitRate": "min"}).withColumn("HitRate", col("min(HitRate)"))

Finally, we use join generate the Closest Def Distance, Shot Clock and shot distance for 4 secific players based on their hit rate. This step involves grouping columns and dropping duplicate values whereever necessary.

In [38]:
df_nba_q2 = df_nba_q2.join(df_nba_q22, ["PlayerID", "HitRate"]).select("PlayerID", "ShotDistance", "ClosestDefDistance", "ShotClock")
df_nba_q2 = df_nba_q2.join(df_nba, [df_nba["_c20"] == df_nba_q2["PlayerID"]]) \
    .withColumn("Player Name", col("_c19")).withColumn("Shot Distance", col("ShotDistance")) \
    .withColumn("Closest Def Distance", col("ClosestDefDistance")).withColumn("Shot Clock", col("ShotClock")) \
    .dropDuplicates(["PlayerID", "Closest Def Distance", "Shot Clock", "Shot Distance"]) \
    .select("Player Name", "Closest Def Distance", "Shot Clock", "Shot Distance")

In [39]:
def bestZoneForPlayer(playerName, df):
    df.where(col("Player Name") == playerName).show()

for names in ["lebron james", "james harden", "chris paul", "stephen curry"]:
    bestZoneForPlayer(names, df_nba_q2)

+------------+--------------------+----------+-------------+
| Player Name|Closest Def Distance|Shot Clock|Shot Distance|
+------------+--------------------+----------+-------------+
|lebron james|                 3.7|      10.3|         19.5|
+------------+--------------------+----------+-------------+

+------------+--------------------+----------+-------------+
| Player Name|Closest Def Distance|Shot Clock|Shot Distance|
+------------+--------------------+----------+-------------+
|james harden|                 2.9|      12.0|         14.1|
+------------+--------------------+----------+-------------+

+-----------+--------------------+----------+-------------+
|Player Name|Closest Def Distance|Shot Clock|Shot Distance|
+-----------+--------------------+----------+-------------+
| chris paul|                 0.4|      null|         22.7|
+-----------+--------------------+----------+-------------+

+-------------+--------------------+----------+-------------+
|  Player Name|Closest De

In this question, we establish the shooting comfort zone for each player as a matrix of "SHOT DIST, CLOSE DEF DIST, SHOT CLOCK." The optimum zone for players like Lebron James, James Harden, Chris Paul, and Stephen Curry is determined by the output's hit rate. The four tables display the closest defensive distance, shot clock, and shooting distance with respect to each of the four players based on the essential parameters and hit rate.

# END OF ASSIGNMENT